In [ ]:
import os, csv, time, struct, pickle
from datetime import datetime
import cv2, numpy as np
import serial

In [ ]:
def crop_and_warp(bgr, mask, M, scale=0.5, ox=-20, oy=15):
    h, w = bgr.shape[:2]
    ch, cw = int(h*scale), int(w*scale)
    cx, cy = w//2 + int(ox*scale), h//2 + int(oy*scale)
    x1, y1 = max(0, cx-cw//2), max(0, cy-ch//2)
    x2, y2 = min(w, x1+cw),     min(h, y1+ch)
    crop = bgr[y1:y2, x1:x2]

    m = cv2.resize(mask, (crop.shape[1], crop.shape[0]), interpolation=cv2.INTER_NEAREST)
    masked = cv2.bitwise_and(crop, crop, mask=m)
    warped = cv2.warpPerspective(masked, M, (275, 275))
    gray   = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    return warped, gray

def normalize_background(image, k=121, background=None, sigma=0):
    if image.ndim == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if k % 2 == 0:
        k += 1
    bg = image if background is None else background
    bg_used = cv2.GaussianBlur(bg, (k, k), sigma)
    img32, bg32 = image.astype(np.float32), bg_used.astype(np.float32)
    eps = 1e-3
    k1 = (1.0 / (bg32.mean() + eps)) * 128.0
    norm = img32 / (bg32 * k1 + eps)
    norm = cv2.normalize(norm, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    return norm

def detect_blobs(normalized_img,
                 detect_black=True,    # True for black dots, False for white
                 min_area=200,          # area lower bound
                 max_area=None,         # area upper bound (auto if None)
                 min_circularity=0.1    # how round it should be
                 ):
    """
    Detect round blobs using SimpleBlobDetector
    """
    img = normalized_img
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype(np.uint8)

    h, w = img.shape[:2]
    if max_area is None:
        max_area = 0.25 * h * w  # auto upper limit

    # setup blob detector params
    params = cv2.SimpleBlobDetector_Params()
    params.filterByColor = True
    params.blobColor = 0 if detect_black else 255
    params.filterByArea = True
    params.minArea = float(min_area)
    params.maxArea = float(max_area)
    params.filterByCircularity = True
    params.minCircularity = float(min_circularity)

    # create detector (opencv version check)
    ver = cv2.__version__.split('.')
    if int(ver[0]) < 3:
        detector = cv2.SimpleBlobDetector(params)
    else:
        detector = cv2.SimpleBlobDetector_create(params)

    # detect blobs
    keypoints = detector.detect(img)

    # draw them
    im_with_keypoints = cv2.drawKeypoints(
        img, keypoints, np.array([]),
        (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    )

    return keypoints, im_with_keypoints


In [ ]:
def process_single_image(bgr, mask, M, 
                         scale=0.5, ox=-20, oy=15, 
                         k=121, background=None, sigma=0, 
                         alpha=1.5, beta=0.0,
                         detect_black=True, min_area=200, max_area=None, min_circularity=0.05):
    """
    Do a quick pipeline on a frame:
    crop and warp
    normalize background
    boost contrast
    find blobs
    """
    # crop ROI and warp
    warped, gray = crop_and_warp(bgr, mask, M, scale, ox, oy)

    # remove background lighting
    norm = normalize_background(gray, k, background, sigma)

    # enhance contrast, smooth a bit
    enhanced = cv2.convertScaleAbs(norm, alpha=alpha, beta=beta)
    enhanced = cv2.GaussianBlur(enhanced, (9, 9), 0)

    # threshold, invert if looking for black dots
    if detect_black:
        _, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    else:
        _, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # clean noise
    opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN,
                              cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15)))
    # close small gaps
    morphed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE,
                               cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))

    # final mask for blob detection
    proc_for_blob = cv2.bitwise_not(morphed) if detect_black else morphed

    # detect blobs
    keypoints, im_with_keypoints = detect_blobs(
        enhanced, detect_black=detect_black,
        min_area=min_area, max_area=max_area,
        min_circularity=min_circularity
    )

    return keypoints, im_with_keypoints, warped


def sort_blobs_simple(keypoints, grid_shape=(5,5), tol_radius=0.5):
    """
    Try to sort blobs into a grid layout
    """
    rows, cols = grid_shape
    expected = rows * cols

    # filter by radius
    if len(keypoints) > expected:
        r = np.array([kp.size/2 for kp in keypoints])
        med = np.median(r)
        lo, hi = med*(1-tol_radius), med*(1+tol_radius)
        keypoints = [kp for kp, ok in zip(keypoints, (r>=lo)&(r<=hi)) if ok]

    # get coords
    pts = np.array([kp.pt for kp in keypoints], np.float32)
    if len(pts) == 0:
        raise ValueError("no keypoints")

    # bin by y, then sort each row by x
    pts = pts[np.argsort(pts[:,1])]
    row_bins = np.array_split(pts, rows)

    selected, leftovers = [], []
    for row_pts in row_bins:
        if len(row_pts)==0:
            selected.append(np.empty((0,2),np.float32))
            continue
        row_pts = row_pts[np.argsort(row_pts[:,0])]
        if len(row_pts)>cols:
            idx = np.linspace(0,len(row_pts)-1,cols).round().astype(int)
            leftovers.append(row_pts[np.setdiff1d(np.arange(len(row_pts)), idx)])
            row_pts = row_pts[idx]
        selected.append(row_pts)

    # flatten
    pts_out = np.vstack([r for r in selected if len(r)>0]) if selected else np.empty((0,2),np.float32)
    return pts_out


In [2]:
class KF2D:
    def __init__(self, x, y, dt=1.0, q=0.02, r=0.8):
        # state: [x, y, vx, vy]
        self.x = np.array([x, y, 0., 0.], dtype=float)
        self.P = np.eye(4) * 20.0
        self.F = np.array([[1,0,dt,0],
                           [0,1,0,dt],
                           [0,0,1,0 ],
                           [0,0,0,1 ]], dtype=float)
        self.H = np.array([[1,0,0,0],
                           [0,1,0,0 ]], dtype=float)
        # process noise (higher → follows movement more)
        self.Q = np.diag([q, q, q*2, q*2]).astype(float)
        # measurement noise (higher → trusts filter more)
        self.R = np.eye(2) * (r**2)

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.x[:2]

    def update(self, z):
        # z: [x, y]
        z = np.asarray(z, dtype=float)
        y = z - (self.H @ self.x)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        I = np.eye(4)
        self.P = (I - K @ self.H) @ self.P
        return self.x[:2]

    def pos(self):
        return self.x[:2].copy()


def _greedy_assign(preds, detections, gate_radius):
    """
    Simple greedy assignment: match predicted to detected points
    """
    N = len(preds)
    M = len(detections)
    match_idx = np.full(N, -1, dtype=int)
    if M == 0 or N == 0:
        return match_idx

    # compute distances
    D = np.linalg.norm(preds[:,None,:] - detections[None,:,:], axis=2)
    D[D > gate_radius] = np.inf

    # sort candidate pairs by distance
    pairs = np.argwhere(np.isfinite(D))
    if pairs.size == 0:
        return match_idx
    costs = D[pairs[:,0], pairs[:,1]]
    order = np.argsort(costs)

    used_det = np.zeros(M, dtype=bool)
    used_trk = np.zeros(N, dtype=bool)
    for k in order:
        i, j = pairs[k]
        if (not used_trk[i]) and (not used_det[j]):
            match_idx[i] = j
            used_trk[i] = True
            used_det[j] = True
    return match_idx


def remove_global_affine(P0, Pt):
    """
    Remove global affine transform between two sets of points
    """
    if len(P0) < 3 or len(Pt) < 3:
        return Pt - P0
    try:
        A, _ = cv2.estimateAffine2D(P0, Pt, ransacReprojThreshold=1.5)
        if A is None:
            return Pt - P0
        Pt_aff = (P0 @ A[:,:2].T) + A[:,2]   # aligned version
        return Pt - Pt_aff                    # residuals after removing global motion
    except:
        return Pt - P0


def grid_divergence(P0, D, rows=5, cols=5):
    """
    Compute divergence field on a rows×cols grid
    """
    U = D[:,0].reshape(rows, cols)
    V = D[:,1].reshape(rows, cols)
    X = P0[:,0].reshape(rows, cols)
    Y = P0[:,1].reshape(rows, cols)

    # estimate grid spacing (median is more robust)
    dx_vals = np.diff(X, axis=1).ravel()
    dy_vals = np.diff(Y, axis=0).ravel()
    dx = np.median(np.abs(dx_vals)) if len(dx_vals) > 0 else 1.0
    dy = np.median(np.abs(dy_vals)) if len(dy_vals) > 0 else 1.0
    if dx <= 0: dx = 1.0
    if dy <= 0: dy = 1.0

    # gradients → divergence
    dUdy, dUdx = np.gradient(U, dy, dx)
    dVdy, dVdx = np.gradient(V, dy, dx)
    return (dUdx + dVdy).ravel()


def gaussian_splat_weighted(centers, weights, H=275, W=275, sigma=8.0):
    """
    Make a weighted gaussian density map
    """
    Dmap = np.zeros((H, W), np.float32)
    if len(centers) == 0 or len(weights) == 0:
        return Dmap

    r = int(3 * sigma)
    yy, xx = np.mgrid[-r:r+1, -r:r+1]
    kernel = np.exp(-(xx**2 + yy**2)/(2*sigma**2)).astype(np.float32)

    for (cx, cy), w in zip(centers, weights):
        x, y = int(round(cx)), int(round(cy))
        x0 = max(0, x-r); x1 = min(W, x+r+1)
        y0 = max(0, y-r); y1 = min(H, y+r+1)
        if x0 >= x1 or y0 >= y1:
            continue

        kx0 = x0 - (x - r); kx1 = kx0 + (x1 - x0)
        ky0 = y0 - (y - r); ky1 = ky0 + (y1 - y0)

        Dmap[y0:y1, x0:x1] += w * kernel[ky0:ky1, kx0:kx1]

    return Dmap


In [3]:
# Serial communication helpers
def _find_valid_frame(buf: bytearray):
    for i in range(len(buf) - 19):
        if buf[i] == 0xAA and buf[i+1] == 0x55 and buf[i+19] == 0xA5:
            return i
    return None

def _is_fresh(sample, now, ttl):
    return (sample is not None) and (now - sample["t"] <= ttl)

def _freeze_ready(latest_dict, now, ttl, require_all, min_fresh):
    fresh_cnt = sum(_is_fresh(latest_dict[sid], now, ttl) for sid in latest_dict)
    total = len(latest_dict)
    if require_all:
        return (fresh_cnt == total), fresh_cnt, total
    else:
        return (fresh_cnt >= min_fresh), fresh_cnt, total

# Main data collection system
def collect_with_tracking_and_density(
    mask,  M,
    save_root="dataset",
    session_name=None,
    cam_index=1, width=640, height=480,
    crop_scale=0.5, crop_offset_x=-20, crop_offset_y=15,
    grid_shape=(5,5), tol_radius=0.2,
    # tracking
    gate_radius_init=14.0, gate_radius_max=32.0,
    q_proc=4, r_meas=0.5, post_ema=0.15,
    lost_to_rebuild=4, severe_lost_ratio=0.35,
    # density map
    density_sigma=8.0,
    use_negative=True,
    # serial
    serial_port="COM7", baud=115200, sensor_ids=(0,1,2,3,4),
    stale_timeout_s=2.0, require_all_sensors=False, min_fresh_sensors=3
):
    """
    End-to-end data collection: blob tracking + density map generation + data saving.
    """

    rows, cols = grid_shape
    expected = rows * cols
    CANVAS = 275

    # Create folders
    session_name = session_name or datetime.now().strftime("%Y%m%d_%H%M%S")
    sess_dir = os.path.join(save_root, session_name)
    img_dir = os.path.join(sess_dir, "images")           # raw images
    density_dir = os.path.join(sess_dir, "density_maps") # 64x64 density maps

    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(density_dir, exist_ok=True)

    # CSV files
    baselines_csv = os.path.join(sess_dir, "baselines.csv")
    labels_csv = os.path.join(sess_dir, "labels.csv")

    if not os.path.exists(baselines_csv):
        with open(baselines_csv, "w", newline="", encoding="utf-8") as f:
            hdr = ["baseline_id", "timestamp"] + \
                  [f"P0x{i+1}" for i in range(expected)] + \
                  [f"P0y{i+1}" for i in range(expected)]
            csv.writer(f).writerow(hdr)

    if not os.path.exists(labels_csv):
        with open(labels_csv, "w", newline="", encoding="utf-8") as f:
            hdr = ["timestamp", "image_file", "density_file", "baseline_id", 
                   "tag", "shape", "size_mm"] + \
                  [f"x{i+1}" for i in range(expected)] + \
                  [f"y{i+1}" for i in range(expected)]
            for sid in sensor_ids:
                hdr += [f"Bx{sid}", f"By{sid}", f"Bz{sid}"]
            csv.writer(f).writerow(hdr)

    # Initialize camera
    cap = cv2.VideoCapture(cam_index, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

    print("Starting camera...")
    time.sleep(2)
    for _ in range(5):
        cap.read()

    # Initialize serial port
    ser = serial.Serial(serial_port, baud, timeout=0.01)
    rx_buf = bytearray()
    latest = {sid: None for sid in sensor_ids}

    def pump_serial():
        data = ser.read(256)
        if data:
            rx_buf.extend(data)
        while len(rx_buf) >= 20:
            pos = _find_valid_frame(rx_buf)
            if pos is None:
                rx_buf[:] = rx_buf[-19:]
                break
            frame_bytes = rx_buf[pos:pos+20]
            del rx_buf[:pos+20]
            sid = frame_bytes[2]
            if sid in latest:
                f0, bx, by, bz = struct.unpack('<ffff', frame_bytes[3:19])
                latest[sid] = {
                    "Bx": float(bx),
                    "By": float(by),
                    "Bz": float(bz),
                    "t": time.time()
                }

    # Tracker state
    P0 = None 
    baseline_id = None
    next_baseline_id = 1

    # If an initial baseline exists, persist it
    if P0 is not None and len(P0) == expected:
        baseline_id = next_baseline_id
        next_baseline_id += 1
        ts = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        with open(baselines_csv, "a", newline="", encoding="utf-8") as f:
            row = [baseline_id, ts] + P0[:,0].tolist() + P0[:,1].tolist()
            csv.writer(f).writerow(row)
        np.save(os.path.join(sess_dir, f"baseline_P0_{baseline_id}.npy"), P0)

    kfs = None
    vis_pts = None
    lost_age = None
    have_init = False
    consec_severe_lost = 0

    # Metadata (tag/shape/size)
    tag_text = ""
    shape_list = ["circle", "square", "rectangle", "triangle", "other"]
    shape_idx = 0
    size_mm = 0.0
    typing_tag = False

    # UI / control flags
    frozen = False
    snapshot = None
    show_density = True

    # Main window
    win = "Data Collection System"
    cv2.namedWindow(win, cv2.WINDOW_NORMAL)

    print("\n" + "="*60)
    print("Data collection system is running")
    print("="*60)
    print("Hotkeys:")
    print("  SPACE: Freeze / unfreeze")
    print("  ENTER: Save current sample (image + density map)")
    print("  BACKSPACE: Discard current frozen sample")
    print("  B: Set a new baseline")
    print("  R: Reset tracking")
    print("  T: Enter tag text")
    print("  C: Toggle shape")
    print("  -/+: Size ±1 mm")
    print("  ,/.: Size ±5 mm")
    print("  D: Show / hide density map")
    print("  N: Toggle negative density")
    print("  Z/X: Adjust sigma")
    print("  ESC: Quit")
    print("="*60)

    try:
        while True:
            # Serial polling
            pump_serial()

            # Grab a frame
            ok, frame = cap.read()
            if not ok:
                cv2.waitKey(10)
                continue

            # Blob detection (expects process_single_image to be available)
            keypoints, im_kp, _ = process_single_image(
                frame, mask, M,
                scale=crop_scale, ox=crop_offset_x, oy=crop_offset_y,
                k=91, background=None, sigma=0,
                alpha=3, beta=0.0,
                detect_black=True, min_area=200, max_area=None, min_circularity=0.05
            )
            det = np.asarray([kp.pt for kp in keypoints], dtype=np.float32) if len(keypoints) > 0 else np.zeros((0,2), np.float32)

            # Prepare canvas for drawing
            canvas = im_kp.copy()
            density_colored = None
            density_raw = None  # raw (float) density map used for saving

            # Tracking logic
            if not have_init:
                # Initialize with a sorted grid if enough blobs are visible
                if len(det) > 0:
                    sorted_xy = sort_blobs_simple(keypoints, grid_shape=grid_shape, tol_radius=tol_radius)
                    if len(sorted_xy) == expected:
                        sorted_xy = sorted_xy.astype(np.float32)
                        kfs = [KF2D(x=sorted_xy[i,0], y=sorted_xy[i,1], dt=1.0, q=q_proc, r=r_meas) 
                               for i in range(expected)]
                        vis_pts = sorted_xy.copy()
                        lost_age = np.zeros(expected, dtype=int)
                        have_init = True
                        consec_severe_lost = 0

                        if P0 is None:
                            P0 = sorted_xy.copy()
                            print("Use current detection as baseline (P0)")

                        for i, (x, y) in enumerate(vis_pts):
                            cv2.circle(canvas, (int(x), int(y)), 6, (0, 255, 0), 2)
                            cv2.putText(canvas, str(i+1), (int(x)+5, int(y)-5),
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                    else:
                        cv2.putText(canvas, f"Initializing... {len(det)}/{expected}", (10, 24),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
            else:
                # Predict / update each KF, greedy associate with detections
                preds = np.stack([kf.predict() for kf in kfs], axis=0)
                gate_each = np.minimum(gate_radius_init * (1.0 + 0.4*lost_age), gate_radius_max)
                gate_use = float(np.max(gate_each)) if len(gate_each) > 0 else gate_radius_init
                match = _greedy_assign(preds, det, gate_radius=gate_use)

                num_matched = 0
                for i in range(expected):
                    j = match[i]
                    if j >= 0:
                        kfs[i].update(det[j])
                        lost_age[i] = 0
                        num_matched += 1
                    else:
                        lost_age[i] += 1

                lost_ratio = float(expected - num_matched) / float(expected)
                if lost_ratio >= severe_lost_ratio:
                    consec_severe_lost += 1
                else:
                    consec_severe_lost = 0

                # Rebuild if tracking collapses and we see enough blobs again
                if consec_severe_lost >= lost_to_rebuild and len(det) >= expected:
                    sorted_xy = sort_blobs_simple(keypoints, grid_shape=grid_shape, tol_radius=tol_radius)
                    if len(sorted_xy) == expected:
                        sorted_xy = sorted_xy.astype(np.float32)
                        kfs = [KF2D(x=sorted_xy[i,0], y=sorted_xy[i,1], dt=1.0, q=q_proc, r=r_meas) 
                               for i in range(expected)]
                        vis_pts = sorted_xy.copy()
                        lost_age[:] = 0
                        consec_severe_lost = 0
                        cv2.putText(canvas, "REBUILD", (10, 24), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

                # EMA smoothing for visualization
                kf_out = np.stack([kf.pos() for kf in kfs], axis=0)
                if vis_pts is None:
                    vis_pts = kf_out.copy()
                else:
                    vis_pts = post_ema * kf_out + (1.0 - post_ema) * vis_pts

                # Density map pipeline (requires the geometry helpers)
                if P0 is not None and vis_pts is not None:
                    try:
                        Pt = vis_pts.astype(np.float32)

                        # 1) Remove global affine motion
                        D = remove_global_affine(P0, Pt)

                        # 2) Compute divergence weights on the grid
                        div_weights = -grid_divergence(P0, D, rows, cols)

                        # 3) Splat into a dense map
                        density_map = gaussian_splat_weighted(Pt, div_weights, H=275, W=275, sigma=density_sigma)

                        density_raw = density_map.copy()

                        # 4) Visualization tweaks
                        if use_negative:
                            density_map = -density_map

                        if density_map.max() != density_map.min():
                            density_map = (density_map - density_map.min()) / (density_map.max() - density_map.min())
                        else:
                            density_map = np.zeros_like(density_map)

                        density_u8 = (density_map * 255).astype(np.uint8)
                        density_colored = cv2.applyColorMap(density_u8, cv2.COLORMAP_JET)

                    except Exception as e:
                        print(f"Density map error: {e}")

                # Draw tracked points
                for i, (x, y) in enumerate(vis_pts):
                    color = (0, 255, 0) if lost_age[i] == 0 else (0, 165, 255)
                    cv2.circle(canvas, (int(x), int(y)), 6, color, 2)
                    cv2.putText(canvas, str(i+1), (int(x)+5, int(y)-5),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

                if consec_severe_lost > 0:
                    cv2.putText(canvas, f"Relinking... {consec_severe_lost}/{lost_to_rebuild}",
                              (10, 24), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)

            # HUD/overlay info
            now = time.time()
            ok_freeze, fresh_cnt, total_cnt = _freeze_ready(latest, now, stale_timeout_s, 
                                                           require_all_sensors, min_fresh_sensors)

            status_text = "FROZEN - Press ENTER to save" if frozen else "LIVE"
            status_color = (50, 230, 50) if frozen else (0, 200, 255)
            cv2.putText(canvas, status_text, (10, CANVAS-30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, status_color, 2)

            sensor_text = f"Sensors: {fresh_cnt}/{total_cnt}"
            cv2.putText(canvas, sensor_text, (10, CANVAS-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 255, 200), 1)

            meta_text = f"{shape_list[shape_idx]} {size_mm:.0f}mm"
            if tag_text:
                meta_text = f"{tag_text} | " + meta_text
            cv2.putText(canvas, meta_text, (10, 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (220, 220, 180), 1)

            # Compose display
            if show_density and density_colored is not None:
                display = np.hstack([canvas, density_colored])
            else:
                display = canvas

            cv2.imshow(win, display)

            # Keyboard handling
            key = cv2.waitKey(1) & 0xFF

            # Tag input mode
            if typing_tag and key not in (255, 27):
                if key == 13:  # Enter
                    typing_tag = False
                    print(f"Tag set: {tag_text}")
                elif key in (8, 127):  # Backspace
                    tag_text = tag_text[:-1]
                elif 32 <= key <= 126:
                    tag_text += chr(key)
                continue

            # Main hotkeys
            if key == 27:  # ESC
                break

            elif key == ord(' '):  # SPACE - freeze/unfreeze
                if not frozen:
                    pts_ready = vis_pts is not None and len(vis_pts) == expected
                    if pts_ready and ok_freeze:
                        frozen = True
                        ts = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                        snapshot = {
                            "timestamp": ts,
                            "Pt": vis_pts.copy(),
                            "P0": P0.copy() if P0 is not None else None,
                            "baseline_id": baseline_id,
                            "density_raw": density_raw.copy() if density_raw is not None else None,
                            "latest": {sid: (latest[sid].copy() if latest[sid] else None) for sid in sensor_ids},
                            "frame": frame.copy()
                        }
                        print("Frozen")
                    else:
                        print("Need 25 points and fresh sensor data")
                else:
                    frozen = False
                    snapshot = None
                    print("Unfrozen")

            elif key == 13:  # ENTER - save current frozen sample
                if frozen and snapshot is not None:
                    ts = snapshot["timestamp"]

                    # Save raw image
                    img_name = f"{ts}.jpg"
                    cv2.imwrite(os.path.join(img_dir, img_name), snapshot["frame"])

                    # Save 64x64 density map
                    density_name = f"{ts}.png"
                    if snapshot["density_raw"] is not None:
                        density_64 = cv2.resize(snapshot["density_raw"], (64, 64), interpolation=cv2.INTER_AREA)
                        if use_negative:
                            density_64 = -density_64
                        if density_64.max() != density_64.min():
                            density_64 = (density_64 - density_64.min()) / (density_64.max() - density_64.min())
                            density_64 = (density_64 * 255).astype(np.uint8)
                        else:
                            density_64 = np.zeros((64, 64), dtype=np.uint8)
                        cv2.imwrite(os.path.join(density_dir, density_name), density_64)

                    # Append label row
                    with open(labels_csv, "a", newline="", encoding="utf-8") as f:
                        wr = csv.writer(f)
                        row = [ts, img_name, density_name, baseline_id, 
                               tag_text, shape_list[shape_idx], size_mm]

                        Pt = snapshot["Pt"]
                        row += Pt[:,0].tolist() + Pt[:,1].tolist()

                        for sid in sensor_ids:
                            s = snapshot["latest"].get(sid)
                            if s is None:
                                row += [np.nan, np.nan, np.nan]
                            else:
                                row += [s["Bx"], s["By"], s["Bz"]]

                        wr.writerow(row)

                    print(f"Saved: {img_name} + {density_name}")
                    frozen = False
                    snapshot = None

            elif key in (8, 127):  # BACKSPACE - discard
                if frozen:
                    frozen = False
                    snapshot = None
                    print("Discarded")

            elif key in (ord('b'), ord('B')):  # new baseline
                if vis_pts is not None and len(vis_pts) == expected:
                    P0 = vis_pts.copy().astype(np.float32)
                    baseline_id = next_baseline_id
                    next_baseline_id += 1
                    ts = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

                    with open(baselines_csv, "a", newline="", encoding="utf-8") as f:
                        row = [baseline_id, ts] + P0[:,0].tolist() + P0[:,1].tolist()
                        csv.writer(f).writerow(row)
                    np.save(os.path.join(sess_dir, f"baseline_P0_{baseline_id}.npy"), P0)
                    print(f"New baseline set: ID={baseline_id}")

            elif key in (ord('r'), ord('R')):  # reset tracking
                have_init = False
                kfs = None
                vis_pts = None
                consec_severe_lost = 0
                print("Tracking reset")

            elif key in (ord('t'), ord('T')):  # tag input
                typing_tag = True
                print("Type tag...")

            elif key in (ord('c'), ord('C')):  # shape cycle
                shape_idx = (shape_idx + 1) % len(shape_list)
                print(f"Shape: {shape_list[shape_idx]}")

            elif key in (ord('d'), ord('D')):  # toggle density overlay
                show_density = not show_density

            elif key in (ord('-'), ord('_')):
                size_mm -= 1.0
            elif key in (ord('='), ord('+')):
                size_mm += 1.0
            elif key == ord(','):
                size_mm -= 5.0
            elif key == ord('.'):
                size_mm += 5.0

            elif key == ord('z'):  # decrease sigma
                density_sigma = max(2.0, density_sigma - 1.0)
                print(f"Density sigma: {density_sigma}")
            elif key == ord('x'):  # increase sigma
                density_sigma = min(30.0, density_sigma + 1.0)
                print(f"Density sigma: {density_sigma}")

    finally:
        cap.release()
        ser.close()
        cv2.destroyAllWindows()
        print("Collection ended")

# Entrypoint
if __name__ == "__main__":
    # Load ROI data
    with open("roi_data.pkl", "rb") as f:
        mask, ordered_pts, M = pickle.load(f)

    # Run
    collect_with_tracking_and_density(
        mask=mask, 
        M=M,
        serial_port="COM7",
        baud=115200,
        cam_index=1,
        width=640,
        height=480,
        grid_shape=(5,5),
        tol_radius=0.2,
        density_sigma=12.0  # tune as needed
    )


FileNotFoundError: [Errno 2] No such file or directory: 'roi_data.pkl'